In [1]:
import control as ct
import numpy as np
import scipy as sc
import pymor

In [2]:
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.models.iosys import SecondOrderModel

In [3]:
n = 100;  # VELICINA SUSTAVA
i1 = 0;   # POZICIJA 1. PRIGUSIVACA
i2 = 1;   # POZICIJA 2. PRIGUSIVACA
v1 = 200; # VISKOZNOST 1. PRIGUSIVACA
v2 = 100; # VISKOZNOST 2. PRIGUSIVACA
p = 0.5;  # MJESOVITA H2 MJERA [0, 1]

In [4]:
M  = np.matrix(np.zeros((n, n)));
K  = np.matrix(np.zeros((n, n)));
B  = np.matrix(np.zeros((n, 5)));
Cp = np.matrix(np.zeros((10, n)));
Cv = np.matrix(np.zeros((10, n)));

B[0:5, 0:5]    = np.matrix(np.diag(range(1, 6)));
Cp[0:10, 45:55] = np.matrix(np.eye(10));
Cv = Cp;

for i in range(n):
    M[i, i] = 200 - 2 * (i + 1) if i < 50 else i + 1 + 50;
    K[i, i] = 200;
    if i > 0:
        K[i, i - 1] = -100;
    
    if i < n - 1:
        K[i, i + 1] = -100;
        
sM = np.sqrt(M);
isM = np.linalg.inv(sM);
E = 0.04 * sM @ np.matrix(sc.linalg.sqrtm(isM @ K @ isM)) * sM;

E[i1, i1] += v1;
E[i2, i2] += v2;

In [40]:
def Mixed_Norm(M, E, K, B, Cv, Cp, p):
    n = M.shape[0];
    Zg    = 1 / n * np.matrix(np.eye(n));
    
    ## -p * B * B.T - (1 - p) * Zg
    ## -B_M * B_M.T
    ## -[(sqrt(p) * B) * (sqrt(p) * B.T) + sqrt(1 - p) * sqrt(Zg) * sqrt
    
    sp    = np.sqrt(p);
    ssp   = np.sqrt(1 - p);
    ##B_M   = sp * B + ssp * sc.linalg.sqrtm(Zg);
    M     = NumpyMatrixOperator(M);
    E     = NumpyMatrixOperator(E);
    K     = NumpyMatrixOperator(K);
    B     = NumpyMatrixOperator(B);
    Cv    = NumpyMatrixOperator(Cv);
    Cp    = NumpyMatrixOperator(Cp);
    
    # Model = SecondOrderModel(M, E, K, B_M, Cv, Cp);
    Model = SecondOrderModel(M, E, K, B, Cv, Cp);
    
    return Model.h2_norm();

In [41]:
def Mixed_H2Norm(M, E, K, B, Cv, Cp, p):
    n    = M.shape[0];
    Zg   = 1 / (2 * n) * np.matrix(np.eye(2 * n));
    invM = np.linalg.inv(M);
    A    = np.block([[np.zeros((n, n)), np.eye(n)],
                  [-invM * K, -invM * E]
                 ]);
    
    Bt   = (1 - p) * Zg + p * np.block([[np.zeros((n, n)), np.zeros((n, n))],
                                        [np.zeros((n, n)), invM @ B @ B.T @ invM]]);
    
    C = sc.linalg.block_diag(Cp, Cv);
    
    X = ct.lyap(A, Bt @ Bt.T);
    print(X.shape)
    print(C.shape)
    pr = C.T @ C @ X;
    return np.sqrt(np.trace(pr));
    

In [44]:
Mixed_H2Norm(M, E, K, B, Cv, Cp, 1)

(200, 200)
(20, 200)


0.011362934400636383

In [43]:
Mixed_Norm(M, E, K, B, Cv, Cp, 0)

0.4748625989581703